In [48]:
import re
import os
import pandas as pd
import numpy as np

from tqdm import tqdm
from unicodedata import normalize

from src.data import cleandata

In [49]:
tqdm.pandas()

/Users/jalvaradoruiz/opt/anaconda3/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


### Carga Data Feather referente a tribunales, juzgados y provincias

In [50]:
df_juzgados_garantia = pd.read_feather("../data/interim/pjud/JuzgadosGarantia_feather")
df_tribunal_oral = pd.read_feather("../data/interim/pjud/TribunalOral_feather")
df_juzgados_letras = pd.read_feather("../data/interim/pjud/JuzgadosLetras_feather")
df_provincias = pd.read_feather("../data/interim/subdere/Provincias_feather")

In [51]:
# Existen dos provincias en el listado.
filtro_provincia_top = df_tribunal_oral[df_tribunal_oral['COMUNA'].str.contains("PROVINCIA", case = False)]
filtro_provincia_top # agrego a este dataframe las provincias a explorar por comunas

,index,REGION,TRIBUNAL,ASIENTO,JUECES,COMUNA,TIPO JUZGADO
40,40,QUINTA REGION DE VALPARAISO,TRIBUNAL DE JUICIO ORAL EN LO PENAL SAN FELIPE,SAN FELIPE,4,PROVINCIA DE SAN FELIPE,ORAL
41,41,QUINTA REGION DE VALPARAISO,TRIBUNAL DE JUICIO ORAL EN LO PENAL LOS ANDES,LOS ANDES,4,PROVINCIA DE LOS ANDES,ORAL


In [52]:
df_tribunal_oral.drop(filtro_provincia_top.index, axis = 0, inplace = True)

#### Analizo Dataframe , ya que acá estan los Juzgados de Letras con Competencia Común, y en algunos casos estos pueden No corresponder a Juzgados Penales. Para saber eso, debo verificar que no existan JG en esas comunas, en el caso de existir debo eliminar registro.


In [53]:
df_juzgados_letras

,index,REGION,TRIBUNAL,ASIENTO,JUECES,COMUNA,TIPO JUZGADO
0,0,REGION DE TARAPACA,JUZGADO DE LETRAS Y GARANTIA POZO ALMONTE,POZO ALMONTE,3,PICA,LETRAS Y GARANTIA
1,1,REGION DE TARAPACA,JUZGADO DE LETRAS Y GARANTIA POZO ALMONTE,POZO ALMONTE,3,POZO ALMONTE,LETRAS Y GARANTIA
2,2,REGION DE TARAPACA,JUZGADO DE LETRAS Y GARANTIA POZO ALMONTE,POZO ALMONTE,3,HUARA,LETRAS Y GARANTIA
3,3,REGION DE TARAPACA,JUZGADO DE LETRAS Y GARANTIA POZO ALMONTE,POZO ALMONTE,3,COLCHANE,LETRAS Y GARANTIA
4,4,REGION DE TARAPACA,JUZGADO DE LETRAS Y GARANTIA POZO ALMONTE,POZO ALMONTE,3,CAMIÑA,LETRAS Y GARANTIA
...,...,...,...,...,...,...,...
228,228,REGION METROPOLITANA DE SANTIAGO,JUZGADO DE LETRAS Y GARANTIA PEÑAFLOR,PEÑAFLOR,1,PADRE HURTADO,LETRAS Y GARANTIA
229,229,REGION METROPOLITANA DE SANTIAGO,JUZGADO DE LETRAS Y GARANTIA MELIPILLA,MELIPILLA,2,PROVINCIA DE MELIPILLA CON EXCEPCION DE CURACAVI,LETRAS Y GARANTIA
230,230,REGION METROPOLITANA DE SANTIAGO,JUZGADO DE LETRAS Y GARANTIA BUIN,BUIN,1,BUIN,LETRAS Y GARANTIA
231,231,REGION METROPOLITANA DE SANTIAGO,JUZGADO DE LETRAS Y GARANTIA BUIN,BUIN,1,PAINE,LETRAS Y GARANTIA


#### elimino datos inconcruentes generados y separo las provincias


In [54]:
filtro_provincia_jl = df_juzgados_letras[df_juzgados_letras['COMUNA'].str.contains("PROVINCIA", case = False)]
filtro_provincia_jl

,index,REGION,TRIBUNAL,ASIENTO,JUECES,COMUNA,TIPO JUZGADO
8,8,REGION DE ANTOFAGASTA,JUZGADO DE LETRAS Y GARANTIA CALAMA,CALAMA,1,PROVINCIA DE EL LOA,LETRAS Y GARANTIA
40,40,REGION DE VALPARAISO,JUZGADO DE LETRAS Y GARANTIA LOS ANDES,LOS ANDES,1,PROVINCIA DE LOS ANDES,LETRAS Y GARANTIA
60,60,REGION DE VALPARAISO,JUZGADO DE LETRAS Y GARANTIA ISLA DE PASCUA,ISLA DE PASCUA,1,PROVINCIA DE ISLA DE PASCUA,LETRAS Y GARANTIA
176,176,REGION DE LOS LAGOS,JUZGADO DE LETRAS Y GARANTIA ANCUD,ANCUD,1,ESTE TRIBUNAL MANTENDRA SU CARACTER DE JUZGADO...,LETRAS Y GARANTIA
187,187,REGION DE AISEN DEL GENERAL CARLOS IBAÑEZ DEL ...,JUZGADO DE LETRAS Y GARANTIA COCHRANE,COCHRANE,1,PROVINCIA DE CAPITAN PRAT,LETRAS Y GARANTIA
191,191,REGION DE MAGALLANES Y ANTARTICA CHILENA,JUZGADO DE LETRAS Y GARANTIA NATALES,NATALES,2,PROVINCIA DE ULTIMA ESPERANZA,LETRAS Y GARANTIA
192,192,REGION DE MAGALLANES Y ANTARTICA CHILENA,JUZGADO DE LETRAS Y GARANTIA PORVENIR,PORVENIR,1,PROVINCIA DE TIERRA DEL FUEGO,LETRAS Y GARANTIA
193,193,REGION DE MAGALLANES Y ANTARTICA CHILENA,JUZGADO DE LETRAS Y GARANTIA CABO DE HORNOS,CABO DE HORNOS,1,PROVINCIA DE LA ANTARTICA CHILENA,LETRAS Y GARANTIA
229,229,REGION METROPOLITANA DE SANTIAGO,JUZGADO DE LETRAS Y GARANTIA MELIPILLA,MELIPILLA,2,PROVINCIA DE MELIPILLA CON EXCEPCION DE CURACAVI,LETRAS Y GARANTIA
232,232,REGION METROPOLITANA DE SANTIAGO,JUZGADO DE LETRAS Y GARANTIA COLINA,COLINA,3,PROVINCIA DE CHACABUCO,LETRAS Y GARANTIA


##### Elimino los que estan mal generados o con datos de Provincias


In [55]:
df_juzgados_letras.drop(filtro_provincia_jl.index, axis = 0, inplace = True)

In [56]:
df_juzgados_letras

,index,REGION,TRIBUNAL,ASIENTO,JUECES,COMUNA,TIPO JUZGADO
0,0,REGION DE TARAPACA,JUZGADO DE LETRAS Y GARANTIA POZO ALMONTE,POZO ALMONTE,3,PICA,LETRAS Y GARANTIA
1,1,REGION DE TARAPACA,JUZGADO DE LETRAS Y GARANTIA POZO ALMONTE,POZO ALMONTE,3,POZO ALMONTE,LETRAS Y GARANTIA
2,2,REGION DE TARAPACA,JUZGADO DE LETRAS Y GARANTIA POZO ALMONTE,POZO ALMONTE,3,HUARA,LETRAS Y GARANTIA
3,3,REGION DE TARAPACA,JUZGADO DE LETRAS Y GARANTIA POZO ALMONTE,POZO ALMONTE,3,COLCHANE,LETRAS Y GARANTIA
4,4,REGION DE TARAPACA,JUZGADO DE LETRAS Y GARANTIA POZO ALMONTE,POZO ALMONTE,3,CAMIÑA,LETRAS Y GARANTIA
...,...,...,...,...,...,...,...
226,226,REGION METROPOLITANA DE SANTIAGO,JUZGADO DE LETRAS Y GARANTIA TALAGANTE,TALAGANTE,1,ISLA DE MAIPO,LETRAS Y GARANTIA
227,227,REGION METROPOLITANA DE SANTIAGO,JUZGADO DE LETRAS Y GARANTIA PEÑAFLOR,PEÑAFLOR,1,PEÑAFLOR,LETRAS Y GARANTIA
228,228,REGION METROPOLITANA DE SANTIAGO,JUZGADO DE LETRAS Y GARANTIA PEÑAFLOR,PEÑAFLOR,1,PADRE HURTADO,LETRAS Y GARANTIA
230,230,REGION METROPOLITANA DE SANTIAGO,JUZGADO DE LETRAS Y GARANTIA BUIN,BUIN,1,BUIN,LETRAS Y GARANTIA


In [57]:
filtro_provincia_jl

,index,REGION,TRIBUNAL,ASIENTO,JUECES,COMUNA,TIPO JUZGADO
8,8,REGION DE ANTOFAGASTA,JUZGADO DE LETRAS Y GARANTIA CALAMA,CALAMA,1,PROVINCIA DE EL LOA,LETRAS Y GARANTIA
40,40,REGION DE VALPARAISO,JUZGADO DE LETRAS Y GARANTIA LOS ANDES,LOS ANDES,1,PROVINCIA DE LOS ANDES,LETRAS Y GARANTIA
60,60,REGION DE VALPARAISO,JUZGADO DE LETRAS Y GARANTIA ISLA DE PASCUA,ISLA DE PASCUA,1,PROVINCIA DE ISLA DE PASCUA,LETRAS Y GARANTIA
176,176,REGION DE LOS LAGOS,JUZGADO DE LETRAS Y GARANTIA ANCUD,ANCUD,1,ESTE TRIBUNAL MANTENDRA SU CARACTER DE JUZGADO...,LETRAS Y GARANTIA
187,187,REGION DE AISEN DEL GENERAL CARLOS IBAÑEZ DEL ...,JUZGADO DE LETRAS Y GARANTIA COCHRANE,COCHRANE,1,PROVINCIA DE CAPITAN PRAT,LETRAS Y GARANTIA
191,191,REGION DE MAGALLANES Y ANTARTICA CHILENA,JUZGADO DE LETRAS Y GARANTIA NATALES,NATALES,2,PROVINCIA DE ULTIMA ESPERANZA,LETRAS Y GARANTIA
192,192,REGION DE MAGALLANES Y ANTARTICA CHILENA,JUZGADO DE LETRAS Y GARANTIA PORVENIR,PORVENIR,1,PROVINCIA DE TIERRA DEL FUEGO,LETRAS Y GARANTIA
193,193,REGION DE MAGALLANES Y ANTARTICA CHILENA,JUZGADO DE LETRAS Y GARANTIA CABO DE HORNOS,CABO DE HORNOS,1,PROVINCIA DE LA ANTARTICA CHILENA,LETRAS Y GARANTIA
229,229,REGION METROPOLITANA DE SANTIAGO,JUZGADO DE LETRAS Y GARANTIA MELIPILLA,MELIPILLA,2,PROVINCIA DE MELIPILLA CON EXCEPCION DE CURACAVI,LETRAS Y GARANTIA
232,232,REGION METROPOLITANA DE SANTIAGO,JUZGADO DE LETRAS Y GARANTIA COLINA,COLINA,3,PROVINCIA DE CHACABUCO,LETRAS Y GARANTIA


#### EN INDEX 230 , DEJARE SOLO PROVINCIA DE MELIPILLA, DESPUES EXCEPSIONAR CURACAVÍ

In [58]:
filtro_provincia_jl.loc[229,"COMUNA"] = 'PROVINCIA DE MELIPILLA'
filtro_provincia_jl.drop(176, inplace = True)

/Users/jalvaradoruiz/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/Users/jalvaradoruiz/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [59]:
filtro_provincia_jl

,index,REGION,TRIBUNAL,ASIENTO,JUECES,COMUNA,TIPO JUZGADO
8,8,REGION DE ANTOFAGASTA,JUZGADO DE LETRAS Y GARANTIA CALAMA,CALAMA,1,PROVINCIA DE EL LOA,LETRAS Y GARANTIA
40,40,REGION DE VALPARAISO,JUZGADO DE LETRAS Y GARANTIA LOS ANDES,LOS ANDES,1,PROVINCIA DE LOS ANDES,LETRAS Y GARANTIA
60,60,REGION DE VALPARAISO,JUZGADO DE LETRAS Y GARANTIA ISLA DE PASCUA,ISLA DE PASCUA,1,PROVINCIA DE ISLA DE PASCUA,LETRAS Y GARANTIA
187,187,REGION DE AISEN DEL GENERAL CARLOS IBAÑEZ DEL ...,JUZGADO DE LETRAS Y GARANTIA COCHRANE,COCHRANE,1,PROVINCIA DE CAPITAN PRAT,LETRAS Y GARANTIA
191,191,REGION DE MAGALLANES Y ANTARTICA CHILENA,JUZGADO DE LETRAS Y GARANTIA NATALES,NATALES,2,PROVINCIA DE ULTIMA ESPERANZA,LETRAS Y GARANTIA
192,192,REGION DE MAGALLANES Y ANTARTICA CHILENA,JUZGADO DE LETRAS Y GARANTIA PORVENIR,PORVENIR,1,PROVINCIA DE TIERRA DEL FUEGO,LETRAS Y GARANTIA
193,193,REGION DE MAGALLANES Y ANTARTICA CHILENA,JUZGADO DE LETRAS Y GARANTIA CABO DE HORNOS,CABO DE HORNOS,1,PROVINCIA DE LA ANTARTICA CHILENA,LETRAS Y GARANTIA
229,229,REGION METROPOLITANA DE SANTIAGO,JUZGADO DE LETRAS Y GARANTIA MELIPILLA,MELIPILLA,2,PROVINCIA DE MELIPILLA,LETRAS Y GARANTIA
232,232,REGION METROPOLITANA DE SANTIAGO,JUZGADO DE LETRAS Y GARANTIA COLINA,COLINA,3,PROVINCIA DE CHACABUCO,LETRAS Y GARANTIA


In [60]:
# Exploro las provincias y extraigo las comunas -> caso Juzgados Letras 
add_comunas_jl = []

for indice in filtro_provincia_jl.index:
    region = filtro_provincia_jl["REGION"][indice]
    tribunal = filtro_provincia_jl["TRIBUNAL"][indice]
    asiento = filtro_provincia_jl["ASIENTO"][indice]
    jueces = filtro_provincia_jl["JUECES"][indice]
    tipo_juzgado = filtro_provincia_jl["TIPO JUZGADO"][indice]
    
    provincia = filtro_provincia_jl["COMUNA"][indice].split("PROVINCIA DE ")
    comunas_de_provincias = df_provincias.loc[df_provincias["Nombre Provincia"] == provincia[1],"Nombre Comuna"]
    
    for var_comuna in range(len(comunas_de_provincias)):
        comuna = comunas_de_provincias.values[var_comuna]
        add_comunas_jl.append([region,tribunal,asiento,jueces,comuna,tipo_juzgado])


df_nuevas_comunas_jl = pd.DataFrame(add_comunas_jl,columns = ['REGION','TRIBUNAL','ASIENTO','JUECES','COMUNA','TIPO JUZGADO'])     
df_juzgados_letras = df_juzgados_letras.append(df_nuevas_comunas_jl, ignore_index = True)

In [61]:
# Exploro las provincias y extraigo las comunas -> Caso Tribunales orales

add_comunas_top = []

for indice in filtro_provincia_top.index:
    region = filtro_provincia_top["REGION"][indice]
    tribunal = filtro_provincia_top["TRIBUNAL"][indice]
    asiento = filtro_provincia_top["ASIENTO"][indice]
    jueces = filtro_provincia_top["JUECES"][indice]
    tipo_juzgado = filtro_provincia_top["TIPO JUZGADO"][indice]
    
    provincia = filtro_provincia_top["COMUNA"][indice].split("PROVINCIA DE ")
    comunas_de_provincias = df_provincias.loc[df_provincias["Nombre Provincia"] == provincia[1],"Nombre Comuna"]
    
    for var_comuna in range(len(comunas_de_provincias)):
        comuna = comunas_de_provincias.values[var_comuna]
        add_comunas_top.append([region,tribunal,asiento,jueces,comuna,tipo_juzgado])


df_nuevas_comunas_top = pd.DataFrame(add_comunas_top,columns = ['REGION','TRIBUNAL','ASIENTO','JUECES','COMUNA','TIPO JUZGADO'])     

df_tribunal_oral = df_tribunal_oral.append(df_nuevas_comunas_top, ignore_index = True)


#### DE ESTE LISTADO DEBO VERIFICAR LOS QUE TIENEN JUZGADOS DE GARANTIAS Y ELIMINAR DE JUZGADOS DE LETRAS, YA QUE SON LOS JUZGADOS DE GARANTÍA QUE OCUPAN ESA FUNCION EN ESAS COMUNAS

In [62]:
df_juzgados_garantia

,level_0,index,REGION,TRIBUNAL,ASIENTO,JUECES,COMUNA,TIPO JUZGADO
0,0,0,PRIMERA REGION DE TARAPACA,JUZGADO DE GARANTIA IQUIQUE,IQUIQUE,7,IQUIQUE,GARANTIA
1,1,1,SEGUNDA REGION DE ANTOFAGASTA,JUZGADO DE GARANTIA TOCOPILLA,TOCOPILLA,1,TOCOPILLA,GARANTIA
2,2,2,SEGUNDA REGION DE ANTOFAGASTA,JUZGADO DE GARANTIA CALAMA,CALAMA,4,CALAMA,GARANTIA
3,3,3,SEGUNDA REGION DE ANTOFAGASTA,JUZGADO DE GARANTIA CALAMA,CALAMA,4,OLLAGUE,GARANTIA
4,4,4,SEGUNDA REGION DE ANTOFAGASTA,JUZGADO DE GARANTIA CALAMA,CALAMA,4,SAN PEDRO DE ATACAMA,GARANTIA
...,...,...,...,...,...,...,...,...
233,233,233,REGION DE ÑUBLE,JUZGADO DE GARANTIA CHILLAN,CHILLAN,4,CHILLAN VIEJO,GARANTIA
234,234,234,REGION DE ÑUBLE,JUZGADO DE GARANTIA YUNGAY,YUNGAY,1,EL CARMEN,GARANTIA
235,235,235,REGION DE ÑUBLE,JUZGADO DE GARANTIA YUNGAY,YUNGAY,1,PEMUCO,GARANTIA
236,236,236,REGION DE ÑUBLE,JUZGADO DE GARANTIA YUNGAY,YUNGAY,1,YUNGAY,GARANTIA


In [63]:
# Acá debo eliminar los juzgados de letras que estan cubiertos por la competencia de un juzgado de Garantía

juzgados_garantias = df_juzgados_garantia['COMUNA'].unique().tolist()
drop_jl = []

for jl in df_juzgados_letras.index:
    if df_juzgados_letras['COMUNA'][jl] in juzgados_garantias:
        drop_jl.append(jl)

df_juzgados_letras.drop(drop_jl, axis = 0, inplace = True)

In [64]:
df_juzgados_letras

,index,REGION,TRIBUNAL,ASIENTO,JUECES,COMUNA,TIPO JUZGADO
0,0.0,REGION DE TARAPACA,JUZGADO DE LETRAS Y GARANTIA POZO ALMONTE,POZO ALMONTE,3,PICA,LETRAS Y GARANTIA
1,1.0,REGION DE TARAPACA,JUZGADO DE LETRAS Y GARANTIA POZO ALMONTE,POZO ALMONTE,3,POZO ALMONTE,LETRAS Y GARANTIA
2,2.0,REGION DE TARAPACA,JUZGADO DE LETRAS Y GARANTIA POZO ALMONTE,POZO ALMONTE,3,HUARA,LETRAS Y GARANTIA
3,3.0,REGION DE TARAPACA,JUZGADO DE LETRAS Y GARANTIA POZO ALMONTE,POZO ALMONTE,3,COLCHANE,LETRAS Y GARANTIA
4,4.0,REGION DE TARAPACA,JUZGADO DE LETRAS Y GARANTIA POZO ALMONTE,POZO ALMONTE,3,CAMIÑA,LETRAS Y GARANTIA
...,...,...,...,...,...,...,...
236,NaN,REGION DE MAGALLANES Y ANTARTICA CHILENA,JUZGADO DE LETRAS Y GARANTIA PORVENIR,PORVENIR,1,PORVENIR,LETRAS Y GARANTIA
237,NaN,REGION DE MAGALLANES Y ANTARTICA CHILENA,JUZGADO DE LETRAS Y GARANTIA PORVENIR,PORVENIR,1,PRIMAVERA,LETRAS Y GARANTIA
238,NaN,REGION DE MAGALLANES Y ANTARTICA CHILENA,JUZGADO DE LETRAS Y GARANTIA PORVENIR,PORVENIR,1,TIMAUKEL,LETRAS Y GARANTIA
239,NaN,REGION DE MAGALLANES Y ANTARTICA CHILENA,JUZGADO DE LETRAS Y GARANTIA CABO DE HORNOS,CABO DE HORNOS,1,CABO DE HORNOS,LETRAS Y GARANTIA


In [65]:
df_juzgados_penales = pd.concat([df_juzgados_garantia,df_juzgados_letras], join = "inner")

In [66]:
df_juzgados_penales.reset_index(inplace = True)


In [67]:
df_juzgados_penales.count()


level_0         345
index           334
REGION          345
TRIBUNAL        345
ASIENTO         345
JUECES          345
COMUNA          345
TIPO JUZGADO    345
dtype: int64

In [68]:
df_tribunal_oral.count()

index           336
REGION          346
TRIBUNAL        346
ASIENTO         346
JUECES          346
COMUNA          346
TIPO JUZGADO    346
dtype: int64

In [69]:
# Verifico cuales comunas no existen en juzgados penales --> Casos especiales

for indice in df_provincias.index:
    if not df_provincias['Nombre Comuna'][indice] in df_juzgados_penales['COMUNA'].unique():
        print(df_provincias['Nombre Comuna'][indice])

ALTO HOSPICIO


In [70]:
# Verifico cuales comunas no existen en Tribunales orales --> Casos especiales

for indice in df_provincias.index:
    if not df_provincias['Nombre Comuna'][indice] in df_tribunal_oral['COMUNA'].unique():
        print(df_provincias['Nombre Comuna'][indice])

In [71]:
# El caso de Alto Hospicio es particular, no aparece en el COT, ya que fue parte de una actualizacion de la Ley y fue sumado como Juzgado
# especial. Debemos agregarlo a Juzgados Penales

alto_hospicio = [('REGION DE TARAPACA','JUZGADO DE LETRAS Y GARANTIA ALTO HOSPICIO','ALTO HOSPICIO','4',
                 'ALTO HOSPICIO','LETRAS Y GARANTIA')]

df_alto_hospicio = pd.DataFrame(alto_hospicio, columns = ['REGION','TRIBUNAL','ASIENTO','JUECES','COMUNA','TIPO JUZGADO']) 
df_juzgados_penales = pd.concat([df_juzgados_penales, df_alto_hospicio], join = "inner")
df_juzgados_penales['JUECES'] = df_juzgados_penales['JUECES'].fillna(0).astype(np.int8)
df_juzgados_penales.reset_index(drop = True)

,REGION,TRIBUNAL,ASIENTO,JUECES,COMUNA,TIPO JUZGADO
0,PRIMERA REGION DE TARAPACA,JUZGADO DE GARANTIA IQUIQUE,IQUIQUE,7,IQUIQUE,GARANTIA
1,SEGUNDA REGION DE ANTOFAGASTA,JUZGADO DE GARANTIA TOCOPILLA,TOCOPILLA,1,TOCOPILLA,GARANTIA
2,SEGUNDA REGION DE ANTOFAGASTA,JUZGADO DE GARANTIA CALAMA,CALAMA,4,CALAMA,GARANTIA
3,SEGUNDA REGION DE ANTOFAGASTA,JUZGADO DE GARANTIA CALAMA,CALAMA,4,OLLAGUE,GARANTIA
4,SEGUNDA REGION DE ANTOFAGASTA,JUZGADO DE GARANTIA CALAMA,CALAMA,4,SAN PEDRO DE ATACAMA,GARANTIA
...,...,...,...,...,...,...
341,REGION DE MAGALLANES Y ANTARTICA CHILENA,JUZGADO DE LETRAS Y GARANTIA PORVENIR,PORVENIR,1,PRIMAVERA,LETRAS Y GARANTIA
342,REGION DE MAGALLANES Y ANTARTICA CHILENA,JUZGADO DE LETRAS Y GARANTIA PORVENIR,PORVENIR,1,TIMAUKEL,LETRAS Y GARANTIA
343,REGION DE MAGALLANES Y ANTARTICA CHILENA,JUZGADO DE LETRAS Y GARANTIA CABO DE HORNOS,CABO DE HORNOS,1,CABO DE HORNOS,LETRAS Y GARANTIA
344,REGION DE MAGALLANES Y ANTARTICA CHILENA,JUZGADO DE LETRAS Y GARANTIA CABO DE HORNOS,CABO DE HORNOS,1,ANTARTICA,LETRAS Y GARANTIA


In [72]:
# Transformo las variables para que queden igual en los 3 datasets

df_juzgados_penales['REGION'] = df_juzgados_penales['REGION'].progress_apply(cleandata.separa_regiones)
df_tribunal_oral['REGION'] = df_tribunal_oral['REGION'].progress_apply(cleandata.separa_regiones)

100%|██████████| 346/346 [00:00<00:00, 152104.52it/s]


In [73]:
df_juzgados_penales['ASIENTO'] = df_juzgados_penales['ASIENTO'].progress_apply(cleandata.transforma_asiento)
df_tribunal_oral['ASIENTO'] = df_tribunal_oral['ASIENTO'].progress_apply(cleandata.transforma_asiento)

100%|██████████| 346/346 [00:00<00:00, 275218.89it/s]


In [74]:
# Transformo valores de Regiones a similar a BBDD Pjud

regiones_subdere = df_provincias["Nombre Región"].unique()
regiones_top = df_tribunal_oral["REGION"].unique()

for region in regiones_subdere:
    for region_top in regiones_top:
        if region_top.find(region) != -1:
            df_provincias['Nombre Región'] = df_provincias['Nombre Región'].replace(region, region_top) 
    
df_provincias['Nombre Región'] = df_provincias['Nombre Región'].progress_apply(cleandata.separa_regiones)

100%|██████████| 346/346 [00:00<00:00, 161301.45it/s]


In [75]:
# Creare dataframe con tribunales ORales- Garantia y Letras 

top =  df_tribunal_oral[['REGION','TRIBUNAL','ASIENTO','COMUNA']] 
jg = df_juzgados_penales[['REGION','TRIBUNAL','ASIENTO','COMUNA']] 

df_listado_tribunales = pd.concat([top, jg], join = "inner")

## GUARDAMOS DATA EN FEATHER

In [76]:
# Reset el index para realizar feather

df_tribunal_oral.reset_index(inplace = True)
df_juzgados_penales.reset_index(inplace = True)
df_listado_tribunales.reset_index(inplace = True)
df_provincias.reset_index(inplace = True)

# Directorio donde se guardaran archivos feather

path_processed = "../data/processed/pjud"
os.makedirs(path_processed, exist_ok = True) 

# Guardamos dataset como archivo feather

df_tribunal_oral.to_feather(f'{path_processed}/TribunalesOrales_feather')
df_juzgados_penales.to_feather(f'{path_processed}/JuzgadosPenales_feather')
df_listado_tribunales.to_feather(f'{path_processed}/ListadoTribunales_feather')

path_processed_subdere = "../data/processed/subdere"
os.makedirs(path_processed_subdere, exist_ok = True) 
df_provincias.to_feather(f'{path_processed_subdere}/DataRegiones_feather')